<a href="https://colab.research.google.com/github/ogyogy/colaboratory/blob/main/job_shop_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job Shop Scheduling Solver based on Quantum Annealing

[論文](https://arxiv.org/pdf/1506.08479.pdf)

## パッケージのインストール

In [ ]:
!pip install openjij pyqubo

## 問題定義

ジョブ $\mathcal{J} = {j_1, j_2, \dots, j_N}$、マシン $\mathcal{M} = {m_1, m_2, \dots, m_M}$ のジョブショップスケジューリング問題を考える。
このとき、ジョブ $j$ は次のように表される。

$\begin{eqnarray}
j_1 &=& \{ O_1 \rightarrow \cdots \rightarrow O_{k_1} \}, \\
j_2 &=& \{ O_{k_1 + 1} \rightarrow \cdots \rightarrow O_{k_2} \}, \\
\dots \\
j_{N} &=& \{ {O_{k_{N-1}+1}} \rightarrow \cdots \rightarrow {O_{k_N}} \}.
\tag{1}
\end{eqnarray}$

ここで、$O$ はジョブ内の作業を表す。

さらに、$i \in \{1, \dots, k_N \}$ とすると、$i$ 番目の作業 $O_i$ について、 $p_i$ は作業にかかる時間、 $q_i$ は作業に使用するマシンの番号を表す。

In [ ]:
# ジョブの数
N = 3
# 作業の数
kN = 9
# マシンの数
M = 3

### 入力サンプル

| |  operation 1  |  operation 2  | operation 3 |
| :---: | :---: | :---: | :---: |
| $j_1$ | $m_1, p = 2$  | $m_2, p = 1$ | $m_3, p = 1$ |
| $j_2$ | $m_3, p = 2$  | $m_1, p = 1$ | $m_2, p = 2$ |
| $j_3$ | $m_2, p = 1$  | $m_1, p = 1$ | $m_3, p = 2$ |

In [ ]:
import numpy as np

J_ = np.array(
    [1, 1, 1, 2, 2, 2, 3, 3, 3]
)

J = np.array([
    (1, 2), (2, 1), (3, 1),  # j1
    (3, 2), (1, 1), (2, 2),  # j2
    (2, 1), (1, 1), (3, 2)   # j3
])

I_ = J[:, 0]

I = []
for m in range(1, M + 1):
    I.append(np.where(I_ == m)[0].tolist())

## QUBO定式化

バイナリ変数を次のように定義する。

$x_{i, t} = \begin{cases}
    1 : \text{operation} \, O_i \, \text{starts at time} \, t, \\
    0 : \text{otherwise.}
\end{cases} \tag{2}$

ここで、 $t$ はジョブが完了するのに許可される最大時間を表すタイムスパン $T$ によって制限される。タイムスパン自体はすべての作業時間の合計によって制限される。

In [ ]:
from pyqubo import Array, Constraint

# 許可される最大時間
T = sum(J[:, 1])

# O_iが時間tで開始されるとき1、そうでないとき0となるバイナリ変数
x = Array.create('x', shape=(kN, T), vartype='BINARY')

### 制約条件



1つの作業は1度のみ実行される制約は次のように表される。

$
\left(
    \sum_t x_{i,t} = 1 \, \text{for each} \, i    
\right)
\rightarrow \sum_i \left(
    \sum_t x_{i, t} - 1
    \right)^2. \tag{3}
$

任意の時間について、1つのマシンで実行できるジョブは1つだけであり、次のように表される。

 $
 \sum_{(i, t, k, t') \in R_m} x_{i, t} x_{k, t'} = 0 \, \text{for each} \, m
 \tag{4}
 $

ここで、

<center>
$\begin{eqnarray}
R_m &=& A_m \cup B_m,
\\
A_m &=& \{ (i,t,k,t') : (i,k) \in I_m \times I_m,
i \neq k, 0 \leq t, t' < T, 0 < t' - t < p_i
\},
\\
B_m &=& \{ (i,t,k,t') : (i,k) \in I_m \times I_m,
i < k, t' = t, p_i > 0, p_j > 0
\}.
\end{eqnarray}$
</center>

$A_m$ は、別の操作 $O_i$ がまだ実行されている場合に操作 $O_j$ が $t'$ で開始することを禁止する制約を表す。これは、$O_i$ が時間 $t$ で開始し、$t' - t$ が $pi$ 未満の場合に発生します。

$B_m$ は、少なくとも1つのジョブの実行時間が0でない限り、2つのジョブを同時に開始できない制約を表す。

最後に、ジョブ内の操作の順序は次のように表される。
これは、連続する操作間の優先順位違反の数のみをカウントする。

$
\sum_{k_{n-1} < i < k_n \\ t + p_i > t'} x_{i, t} x_{i+1, t'} \, \text{for each} \, n
\tag{5}
$

ハミルトニアンは次のように表される。

$\begin{eqnarray}
H_T(\bar{x}) &=& \eta h_1(\bar{x}) + \alpha h_2(\bar{x}) + \beta h_3(\bar{x}),\tag{6}
\end{eqnarray}$

ここで、

$\begin{eqnarray}
h_1(\bar{x}) &=& \sum_n \left(
    \sum_{k_{n-1} < i < k_n \\ t + p_i > t'} x_{i, t} x_{i+1, t'}
    \right), \tag{7}
\\
h_2(\bar{x}) &=& \sum_m \left(
    \sum_{(i, t, k, t') \in R_m} x_{i, t} x_{k, t'}
    \right), \tag{8}
\\
h_3(\bar{x}) &=& \sum_i \left(
    \sum_t x_{i, t} - 1
    \right)^2, \tag{9}
\end{eqnarray}$

ここで、$\eta, \alpha, \beta > 0$ はハイパーパラメータを表す。

In [ ]:
from itertools import combinations, product

# (7)
# 作業を順番に行う
h1_ = []
for n in range(1, N + 1):
    for i in (np.where(J_ == n)[0][:-1].tolist()):
        for t in range(T):
            for t_ in range(min(t + J[i][1], T)):
                h1_ .append(x[i, t] * x[i + 1, t_])
h1 = Constraint(sum(h1_), label='h1')

# (8)
# 1つのマシンで実行できるジョブは1つのみ
h2_ = []
for m in range(M):
    for i1, i2 in combinations(I[m], 2):
        for t in range(T):
            for t_ in range(t, min(t + J[i1][1], T)):
                h2_ .append(x[i1, t] * x[i2, t_])
h2 = Constraint(sum(h2_), label='h2')

# (9)
# 1つの作業は1度のみ実行される
h3 = Constraint(sum((sum(x[i, t] for t in range(T)) - 1) ** 2 for i in range(kN)), label='h3')

# ハイパーパラメータ
eta = 10
alpha = 10
beta = 10

# (6)
H = (eta * h1) + (alpha * h2) + (beta * h3)

# モデルをコンパイル
model = H.compile()
qubo, offset = model.to_qubo()

In [ ]:
import openjij as oj

# SQAを用いる
sampler = oj.SQASampler()
# SamplerにQUBOを渡す
response = sampler.sample_qubo(Q=qubo, num_reads=300)
# エネルギーが一番低い状態を取得
dict_solution = response.first.sample
# 得られた結果をデコード    
decoded_sample = model.decode_sample(dict_solution, vartype='BINARY')
# 破られている制約
broken = decoded_sample.constraints(only_broken=True)
print('broken: {}'.format(broken))

# ジョブごとに出力
# 縦軸に作業、横軸に時間をとる
print('==========for each job==========')
kn = 0
for n in range(1, N + 1):
    print('----------j_{}----------'.format(n))
    for i in (np.where(J_ == n)[0].tolist()):
        tmp = [decoded_sample.array('x', (i, t)) for t in range(T)]
        print('O_{}: {}'.format(i + 1, ', '.join(list(map(str, tmp)))))

# マシンごとに出力
# 縦軸にマシン、横軸に時間をとる
print('==========for each machine==========')
A = np.zeros((M, T), dtype=int)
for i in range(kN):
    j = J[i]
    for t in range(T):
        flag = decoded_sample.array('x', (i, t))
        if flag == 1:
            try:
                m = J[i][0]
                A[m - 1, t] = J_[i]
                for k in range(J[i][1]):
                    A[m - 1, t + k] = J_[i]
            except IndexError as e:
                print('作業{}が制限時間{}を超えています'.format(i + 1, T))
for m in range(M):
    print('m_{}: {}'.format(m + 1, ', '.join(list(map(str, A[m])))))

broken: {}
==========for each job==========
----------j_1----------
O_1: 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
O_2: 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0
O_3: 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0
----------j_2----------
O_4: 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
O_5: 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
O_6: 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0
----------j_3----------
O_7: 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
O_8: 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0
O_9: 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0
==========for each machine==========
m_1: 0, 1, 1, 0, 0, 2, 0, 3, 0, 0, 0, 0, 0
m_2: 0, 0, 0, 0, 1, 3, 0, 2, 2, 0, 0, 0, 0
m_3: 2, 2, 0, 0, 0, 0, 0, 0, 3, 3, 0, 1, 0
